In [9]:
import wandb
import pandas as pd

In [10]:
api = wandb.Api()
runs_api = api.runs("yort/thesis")

In [148]:
collected_run_names = []
runs = []
for run in runs_api:
    if run.config['epochs'] == 50:
        continue
    if run.name in collected_run_names:
        continue
    collected_run_names.append(run.name)
    run_config = {
        "group": run.group,
        "epochs": run.config['epochs'],
        "metric": run.config['metric'],
        "conv_type": run.config['conv_type'],
        "depth": run.config['backbone_config']['depth'],
        "mlp_depth": run.config['backbone_config']['mlp_depth'],
        "dim": run.config['backbone_config']['hidden_dim'],
    }
    run_results = {
        "balanced_accuracy": run.summary["balanced_accuracy/test"] if "balanced_accuracy/test" in run.summary else -1,
        "map_at_k": run.summary["map_at_k/test"] if "map_at_k/test" in run.summary else -1,
    }
    runs.append({**run_config, **run_results})
# runs.pop(247)
runs = pd.DataFrame(runs)

In [149]:
runs.groupby("conv_type").max("balanced_accuracy")

,epochs,depth,mlp_depth,dim,balanced_accuracy,map_at_k
conv_type,,,,,,
gat,100,2,2,256,0.739593,0.186209
gcn,100,2,2,256,0.728200,0.187613
gin,100,2,2,256,0.722874,0.174065
res,100,2,2,256,0.750703,0.186395


In [150]:
runs.groupby("conv_type").max("map_at_k")

,epochs,depth,mlp_depth,dim,balanced_accuracy,map_at_k
conv_type,,,,,,
gat,100,2,2,256,0.739593,0.186209
gcn,100,2,2,256,0.728200,0.187613
gin,100,2,2,256,0.722874,0.174065
res,100,2,2,256,0.750703,0.186395


## Analyze effect of depth

In [165]:
metric = 'balanced_accuracy'
runs_depth = runs.copy()
runs_depth['conv_type__dim'] = runs_depth.apply(lambda row: str(row.conv_type) + str(row.dim), axis=1)
depth1runs = runs_depth.query(f"{metric} > 0 and depth == 1").sort_values(by=['conv_type__dim', metric])
depth2runs = runs_depth.query(f"{metric} > 0 and depth == 2").sort_values(by=['conv_type__dim', metric])
differences = depth2runs[metric].values - depth1runs[metric].values
differences

array([-0.01016112,  0.00165707,  0.00364844,  0.00320191,  0.00527015,
       -0.00259765, -0.01063619, -0.0131451 , -0.01450015, -0.01642827,
       -0.01752724, -0.01721565, -0.08671358, -0.08014808, -0.07849192,
       -0.06639958, -0.07522854, -0.0726088 , -0.00154413, -0.00167477,
        0.00026499, -0.02908696, -0.01219119, -0.01222385])

In [166]:
from scipy.stats import ttest_1samp as ttest

In [167]:
print("Is depth 2 accuracy less?")
ttest(differences, 0, alternative='less')

Is depth 2 accuracy less?


TtestResult(statistic=-3.9102034612725003, pvalue=0.00035145207753474266, df=23)

## Analyze effect of dim

In [169]:
metric = 'balanced_accuracy'
runs_dim = runs.copy()
runs_dim['conv_type__depth'] = runs_dim.apply(lambda row: str(row.conv_type) + str(row.depth), axis=1)
dim64runs = runs_dim.query(f"{metric} > 0 and dim == 64").sort_values(by=['conv_type__depth', metric])
dim256runs = runs_dim.query(f"{metric} > 0 and dim == 256").sort_values(by=['conv_type__depth', metric])
differences = dim256runs[metric].values - dim64runs[metric].values
differences

array([ 0.01003313,  0.00921452,  0.00123585, -0.0033299 ,  0.00560144,
        0.00748194,  0.00508764,  0.00573048,  0.00763728,  0.01087972,
        0.01011262,  0.01035278,  0.0248775 ,  0.01447519,  0.01486327,
        0.0045635 ,  0.00955565,  0.00898015,  0.00744963,  0.00698258,
        0.00531289,  0.03499246,  0.017499  ,  0.01780174])

In [170]:
print("Is dim 256 accuracy greater?")
ttest(differences, 0, alternative='greater')

Is dim 256 accuracy greater?


TtestResult(statistic=6.466832013105744, pvalue=6.708074197050342e-07, df=23)